In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("/raid/infolab/nlokesh/dataset-interfaces")
import sys
sys.path.append("/raid/infolab/nlokesh/dataset-interfaces")

device = "cuda:5"

In [16]:
from src import main_helper as mh
import constants as constants
from src import models
from pathlib import Path
from dataset_interfaces import imagenet_utils as imu
from tqdm import tqdm
import torch
from utils import torch_utils as tu
import pandas as pd
import numpy as np

# Cls testing given folder name

In [4]:
cls_model = mh.get_model(model_name="resnet50", pretrn=True)
syn_dir = Path("/raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data")
dirs = list(syn_dir.glob("*"))
print(dirs)

/raid/infolab/nlokesh/anaconda3/envs/simrec/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/raid/infolab/nlokesh/anaconda3/envs/simrec/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[PosixPath('/raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/NTI_rain'), PosixPath('/raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/P2P_snow'), PosixPath('/raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/NTI_dusk'), PosixPath('/raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/P2P_sunlight'), PosixPath('/raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/NTI_night'), PosixPath('/raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/NTI_sunlight'), PosixPath('/raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/P2P_rain'), PosixPath('/raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/NTI_base'), PosixPath('/raid/infolab/nlokesh/prana

In [5]:
def evaluate_dir(dir):
    """Evaluates the cls model on the given directory

    Args:
        dir (_type_): _description_
    """
    ds, dl = mh.get_cls_ds(data_dir=dir, loader=True)

    pbar = tqdm(dl, total=len(dl))
    cls_model.eval()
    cls_model.to(device, dtype=torch.float)
    acc_meter = tu.AccuracyMeter(track=["acc"])
    acc_meter.reset()
    cnfs = []

    with torch.no_grad():
        for index, path, sample, target in pbar:
            sysnet = [Path(p).parent.name for p in path]
            cls_ids = [imu.sysnet_to_clsid[sys] for sys in sysnet]
            cls_ids = torch.tensor(cls_ids).to(device, dtype=torch.long)
            sample = sample.to(device, dtype=torch.float)
            logits = cls_model(sample)
            
            preds = torch.argmax(logits, dim=1)
            
            cnf = torch.nn.functional.softmax(logits, dim=1)
            cnf, _ = torch.max(cnf, dim=1)
            cnfs.append(cnf)
            acc_meter.update(y_preds=preds, y=cls_ids)
    print(f"Dir: {dir} Accuracy: {acc_meter.accuracy()} :: avg cnf: {torch.mean(torch.cat(cnfs)).item()}")

for dir in dirs:
    evaluate_dir(dir)

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:03<00:00,  1.25it/s]


Dir: /raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/NTI_rain Accuracy: 0.6 :: avg cnf: 0.6916853189468384


100%|██████████| 2/2 [00:00<00:00,  3.17it/s]


Dir: /raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/P2P_snow Accuracy: 0.62 :: avg cnf: 0.7329110503196716


...

Dir: /raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/NTI_snow Accuracy: 0.56 :: avg cnf: 0.6816549301147461


100%|██████████| 2/2 [00:00<00:00,  3.17it/s]


Dir: /raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/P2P_base Accuracy: 0.8 :: avg cnf: 0.798870325088501


100%|██████████| 2/2 [00:00<00:00,  3.03it/s]

Dir: /raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/P2P_forest Accuracy: 0.64 :: avg cnf: 0.7440425157546997


# Get the Dataset Interfaces Accuracies

In [19]:
cache_dir = Path("/raid/infolab/nlokesh/dataset-interfaces/cache")
shifts = [constants.BASE, constants.RAIN, constants.SNOW, constants.DUSK, constants.SUNLIGHT, constants.FOG]
target_sysnets = Path("/raid/infolab/nlokesh/pranava/recourse-counterfactual-simulator/prompt-to-prompt/synthetic_data/NTI_base").glob("*")
target_sysnets = [Path(p).name for p in target_sysnets]

print(target_sysnets)

for shift in shifts:
    df = pd.read_csv(cache_dir / f"{shift}_preds.csv")
    image_files = df["image_files"].values

    sysnet = [Path(p).parent.name for p in image_files]

    filter_rows = [idx for idx, sys in enumerate(sysnet) if sys in target_sysnets]
    df = df.iloc[filter_rows]

    acc_meter = tu.AccuracyMeter(track=["acc"])
    acc_meter.reset()
    acc_meter.update(y_preds=torch.Tensor(df["pred_y"].values), y=torch.Tensor(df["true_y"].values))

    cnf = df["cnf"].values
    cnf = np.mean(cnf)
    
    print(f"Shift: {shift} Accuracy: {acc_meter.accuracy()} :: avg cnf: {cnf}")
    


['n01734418', 'n01514668', 'n01560419', 'n01882714', 'n01770393', 'n01644373', 'n01630670', 'n01944390', 'n01697457', 'n01806143']
Shift: base Accuracy: 0.898 :: avg cnf: 0.834562438
Shift: in_the_rain Accuracy: 0.914 :: avg cnf: 0.8467809519999999
Shift: in_the_snow Accuracy: 0.888 :: avg cnf: 0.8187982819999999
Shift: at_dusk Accuracy: 0.9 :: avg cnf: 0.8188759800000001
Shift: in_bright_sunlight Accuracy: 0.892 :: avg cnf: 0.82298944
Shift: in_the_fog Accuracy: 0.856 :: avg cnf: 0.7828164300000001
